+ This notebook is part of lecture 26 *Complex matrices and the fast Fourier transform* in the OCW MIT course 18.06 by Prof Gilbert Strang [1]
+ Created by me, Dr Juan H Klopper
    + Head of Acute Care Surgery
    + Groote Schuur Hospital
    + University Cape Town
    + <a href="mailto:juan.klopper@uct.ac.za">Email me with your thoughts, comments, suggestions and corrections</a> 
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/InteractiveResource" property="dct:title" rel="dct:type">Linear Algebra OCW MIT18.06</span> <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">IPython notebook [2] study notes by Dr Juan H Klopper</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

+ [1] <a href="http://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/index.htm">OCW MIT 18.06</a>
+ [2] Fernando Pérez, Brian E. Granger, IPython: A System for Interactive Scientific Computing, Computing in Science and Engineering, vol. 9, no. 3, pp. 21-29, May/June 2007, doi:10.1109/MCSE.2007.53. URL: http://ipython.org

In [ ]:
from IPython.core.display import HTML, Image
css_file = 'style.css'
HTML(open(css_file, 'r').read())

In [ ]:
from sympy import init_printing, Matrix, symbols, I, sqrt, Rational
from IPython.display import Image
from warnings import filterwarnings

In [ ]:
init_printing(use_latex = 'mathjax')
filterwarnings('ignore')

# Complex vectors, matrices
# Fast Fourier transform

## Complex vectors

* Consider the following vector with complex entries (from this point on I will not use the underscore to indicate a vector, so as not to create confusion with the bar, noting complex conjugate, instead, inferring from context)
$$ {z} = \begin{bmatrix} {z}_{1} \\ {z}_{2} \\ \vdots \\ {z}_{n} \end{bmatrix} $$

* The length (actually length squared) of this vector is *no good*, since it should be positive
$$ {z}^{T}{z} $$
* Instead we consider the following
$$ z\bar { z } ={ \left| { z } \right|  }^{ 2 }\\ \therefore \quad \bar { z } ^{ T }z\\ \left[ { \bar { z }  }_{ 1 },{ \bar { z }  }_{ 2 },\dots ,{ \bar { z }  }_{ n } \right] \begin{bmatrix} { z }_{ 1 } \\ { z }_{ 2 } \\ \vdots  \\ { z }_{ n } \end{bmatrix} $$

In [ ]:
z = Matrix([1, I]) # I is the sympy symbol for the imaginary number i
z

* Let's calculate this manually

In [ ]:
z.norm() # The length of a vector

In [ ]:
z_cc = Matrix([1, -I])
z_cc

In [ ]:
sqrt(z_cc.transpose() * z)

* Taking the transpose of the complex conjugate is called the Hermitian
$$ {z}^{H}{z} $$

* We can use the Hermitian for non-complex (or mixed complex) vectors **u** and **v** too
$$ \bar{y}^{T}{x} \\ {y}^{H}{x} $$

In [ ]:
from sympy.physics.quantum.dagger import Dagger # A fun way to quickly get the Hermitian

In [ ]:
Dagger(z)

In [ ]:
sqrt(Dagger(z) * z)

## Complex symmetric matrices

### The transpose

* If the symmetric matrix has complex entries then A<sup>T</sup>=A is *no good*

In [ ]:
A = Matrix([[2, 3 + I], [3 - I, 5]])
A # A Hermitian matrix

In [ ]:
A.transpose() == A

In [ ]:
Dagger(A)

In [ ]:
Dagger(A) == A

* This will work for real-values symmetric matrices as well

In [ ]:
A = Matrix([[3, 4], [4, 2]])
A

In [ ]:
A.transpose() == A

In [ ]:
Dagger(A) == A

### The eigenvalues and eigenvectors

* Back to the complex matrix A

In [ ]:
A = Matrix([[2, 3 + I], [3 - I, 5]])
A

In [ ]:
A.eigenvals()

$$ A=\begin{bmatrix} 2 & 3+i \\ 3-i & 5 \end{bmatrix}\\ A-\lambda I=\underline { 0 } \\ \left| \begin{bmatrix} 2 & 3+i \\ 3-i & 5 \end{bmatrix}-\begin{bmatrix} \lambda  & 0 \\ 0 & \lambda  \end{bmatrix} \right| =0\\ \begin{vmatrix} 2-\lambda  & 3+i \\ 3-i & 5-\lambda  \end{vmatrix}=0\\ \left( 2-\lambda  \right) \left( 5-\lambda  \right) -\left( 3+i \right) \left( 3-i \right) =0\\ 10-7\lambda +{ \lambda  }^{ 2 }-\left( 9+1 \right) =0\\ { \lambda  }^{ 2 }-7\lambda =0\\ { \lambda  }_{ 1 }=0\\ { \lambda  }_{ 2 }=7 $$

In [ ]:
A.eigenvects()

In [ ]:
S, D = A.diagonalize()

In [ ]:
S

In [ ]:
D

* What about S now?
* We have to use its transpose, but it is complex, so we have to take the Hermitian

In [ ]:
Dagger(S)

In [ ]:
S == Dagger(S) # Don't get confused here, S is not symmetric

* Remember that for a symmetric matrix the column vectors in S (usually called Q, the matrix of eigenvectors) are orthogonal, with Q<sup>T</sup>Q=I
* With complex entries we have to consider the Hermitian here, not just the simple transpose
* Here we call Q *unitary*

## The fast Fourier transform

* Look at this special matrix (where we start counting rows and columns at zero)

$$ { F }_{ n }=\begin{bmatrix} W^{ \left( 0 \right) \left( 0 \right)  } & { W }^{ \left( 0 \right) \left( 1 \right)  } & { W }^{ \left( 0 \right) \left( 2 \right)  } & \dots  & { W }^{ \left( 0 \right) \left( n-1 \right)  } \\ W^{ \left( 1 \right) \left( 0 \right)  } & { W }^{ \left( 1 \right) \left( 1 \right)  } & { W }^{ \left( 1 \right) \left( 2 \right)  } & \dots  & { W }^{ \left( 1 \right) \left( n-1 \right)  } \\ { W }^{ \left( 2 \right) \left( 0 \right)  } & { W }^{ \left( 2 \right) \left( 1 \right)  } & { W }^{ \left( 2 \right) \left( 2 \right)  } & \dots  & { W }^{ \left( 2 \right) \left( n-1 \right)  } \\ \vdots  & \vdots  & \vdots  & \dots  & \vdots  \\ { W }^{ \left( n-1 \right) \left( 0 \right)  } & { W }^{ \left( n-1 \right) \left( 1 \right)  } & { W }^{ \left( n-1 \right) \left( 2 \right)  } & \dots  & { W }^{ \left( n-1 \right) \left( n-1 \right)  } \end{bmatrix} \\ \left({F}_{n}\right)_{ij}={W}^{ij}; i,j=0,1,2,\dots,n-1 $$

* W is a special number whose *n*<sup>th</sup> power equals 1
$$ {W}^{n}=1 \\ W={ e }^{ \frac { i2\pi  }{ n }  }=\cos { \frac { 2\pi  }{ n } +i\sin { \frac { 2\pi  }{ n }  }  }  $$
* It is in the complex plane of course (as written in *sin* and *cos* above)

* Remember than *n* here refers to the size the matrix
* Here it also refers to the *n*<sup>th</sup> *n* roots (if that makes any sense, else look at the image below)

In [ ]:
Image(filename = 'W.png')

* So for *n*=4 we will have the following
$$ { F }_{ 4 }=\begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 1 } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 2 } } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 3 } } \\ 1 & \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 2 } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 4 } } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 6 } } \\ 1 & \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 3 } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 6 } } & { \left( { e }^{ \frac { 2\pi i }{ 4 }  } \right) ^{ 9 } } \end{bmatrix} $$

* We note that a quarter of the way around is *i*
$$ {e}^{\frac{2\pi{i}}{4}}={i} $$
* We thus have the following
$$ { F }_{ 4 }=\begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & i & { i }^{ 2 } & { i }^{ 3 } \\ 1 & { i }^{ 2 } & { i }^{ 4 } & { i }^{ 6 } \\ 1 & { i }^{ 3 } & { i }^{ 6 } & { i }^{ 9 } \end{bmatrix}\\ { F }_{ 4 }=\begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & i & -1 & -i \\ 1 & -1 & 1 & -1 \\ 1 & -i & -1 & i \end{bmatrix} $$

* Note how the columns are orthogonal

In [ ]:
F = Matrix([[1, 1, 1, 1], [1, I, -1, -I], [1, -1, 1, -1], [1, -I, -1, I]])
F

In [ ]:
F.col(0) # Calling only the selected column (counting starts at 0)

* The columns are supposed to be orthogonal, i.e. inner (dot) product should be zero
* Clearly below it is not

In [ ]:
F.col(1).dot(F.col(3))

* Remember, though, that this is a complex matrix and we have to use the Hermitian

In [ ]:
col1 = F.col(1)
col3 = F.col(3)
col1, col3

In [ ]:
Dagger(col3), col1

In [ ]:
Dagger(col3) * col1 # Another way to do the dot product

* So, these columns are all orthogonal, but they are not orthonormal
* Note, though that the are all of length 2, so we can normalize each

In [ ]:
Rational(1, 2) * F

* We also note the following
$$ {F}_{n}^{H}{F}_{n}={I} $$
* Just remember to normalize them

In [ ]:
Dagger(Rational(1, 2) * F)

In [ ]:
Dagger(Rational(1, 2) * F) * ((Rational(1, 2) * F))

* Now why do we call it *fast* Fourier transform
* Note the following
$$ { W }_{ n }={ e }^{ \frac { 2\pi i }{ n }  }\\ { \left( { W }_{ n } \right)  }^{ p }={ \left( { e }^{ \frac { 2\pi i }{ n }  } \right)  }^{ p }\\ { \left( { W }_{ 64 } \right)  }^{ 2 }={ \left( { e }^{ \frac { 2\pi i }{ 64 }  } \right)  }^{ 2 };\quad n=64,\quad p=2\\ \therefore \quad { \left( { W }_{ 64 } \right)  }^{ 2 }={ W }_{ 32 } $$

* Now we have the following connection between the two
$$ \left[ { F }_{ 64 } \right] =\begin{bmatrix} I & D \\ I & -D \end{bmatrix}\begin{bmatrix} { F }_{ 32 } & 0 \\ 0 & { F }_{ 32 } \end{bmatrix}\left[ P \right] \\ D=\begin{bmatrix} 1 & 0 & 0 & \dots  & 0 \\ 0 & W & 0 & \dots  & 0 \\ 0 & 0 & { W }^{ 2 } & \dots  & 0 \\ \vdots  & \vdots  & \vdots  & \dots  & \vdots  \\ 0 & 0 & 0 & \dots  & { W }^{ 31 } \end{bmatrix}$$
* P is a permutation matrix

* Going down to 16 will include the following
$$ \begin{bmatrix} I & D & 0 & 0 \\ I & -D & 0 & 0 \\ 0 & 0 & I & D \\ 0 & 0 & I & -D \end{bmatrix}\begin{bmatrix} { F }_{ 16 } & 0 & 0 & 0 \\ 0 & { F }_{ 16 } & 0 & 0 \\ 0 & 0 & { F }_{ 16 } & 0 \\ 0 & 0 & 0 & { F }_{ 16 } \end{bmatrix}\left[ P \right]  $$

* The recursive work above leads to decreasing the work that is required for working with these problems